## Crawling

In [1]:
#crawling website hewania pada bagian blog, https://hewania.com/blog

import aiohttp
import asyncio
from bs4 import BeautifulSoup
import time

# Fungsi asynchronous untuk mengambil URL dari halaman tertentu
async def fetch_page(session, url, scraped_urls, max_links):
    async with session.get(url) as response:
        content = await response.text()
        soup = BeautifulSoup(content, 'html.parser')
        
        # Mencari elemen <h3> dengan kelas 'jeg_post_title'
        articles = soup.find_all('div', class_='btMediaBox')

        new_urls = []
        for article in articles:
            # Mendapatkan elemen <a> di dalam <h3> artikel
            article_link = article.find('a', href=True)
            if article_link:
                article_url = article_link['href']
                
                # Tambahkan URL jika belum di-scrape dan memenuhi jumlah link yang diminta
                if article_url not in scraped_urls and len(scraped_urls) < max_links:
                    scraped_urls.add(article_url)
                    new_urls.append(article_url)

        return new_urls

# Fungsi utama asynchronous untuk mengambil URL dari halaman kategori dan menyimpannya ke file
async def get_links_for_category(session, base_url, max_links, output_file):
    page_num = 1
    scraped_urls = set()
    total_links = 0

    with open(output_file, 'w', encoding='utf-8') as file:
        start_time = time.time()
        while total_links < max_links:
            url = f"{base_url}/page/{page_num}/"  # Navigasi antar halaman
            urls = await fetch_page(session, url, scraped_urls, max_links)

            # Hentikan jika tidak ada artikel ditemukan di halaman
            if not urls:
                print(f"Tidak ada artikel ditemukan di halaman {page_num}. Menghentikan pencarian.")
                break

            for article_url in urls:
                if total_links >= max_links:
                    break
                file.write(article_url + '\n')
                total_links += 1
                print(f"Link terkumpul di {output_file}: {total_links}")

            print(f"Halaman {page_num} untuk {output_file} selesai.")
            page_num += 1
            await asyncio.sleep(1)  # Delay kecil untuk menghindari overload

        end_time = time.time()
        duration = end_time - start_time
        print(f"Waktu yang dibutuhkan untuk {output_file}: {duration:.2f} detik.")

# Fungsi utama untuk mengambil URL dari kategori secara paralel
async def get_links_async(base_url, max_links):
    async with aiohttp.ClientSession() as session:
        output_file = "hewania_links.txt"
        await get_links_for_category(session, base_url, max_links, output_file)

# Fungsi untuk menjalankan asyncio di Jupyter Notebook atau lingkungan dengan event loop
async def main():
    base_url = "https://hewania.com/blog"
    max_links = 3000
    
    await get_links_async(base_url, max_links)

# Memanggil fungsi menggunakan event loop yang sedang berjalan
if __name__ == "__main__":
    try:
        await main()  # Untuk Jupyter atau lingkungan dengan event loop yang aktif
    except RuntimeError:  
        asyncio.run(main())  # Untuk lingkungan tanpa event loop


Link terkumpul di hewania_links.txt: 1
Link terkumpul di hewania_links.txt: 2
Link terkumpul di hewania_links.txt: 3
Link terkumpul di hewania_links.txt: 4
Link terkumpul di hewania_links.txt: 5
Link terkumpul di hewania_links.txt: 6
Link terkumpul di hewania_links.txt: 7
Link terkumpul di hewania_links.txt: 8
Link terkumpul di hewania_links.txt: 9
Halaman 1 untuk hewania_links.txt selesai.
Link terkumpul di hewania_links.txt: 10
Link terkumpul di hewania_links.txt: 11
Link terkumpul di hewania_links.txt: 12
Link terkumpul di hewania_links.txt: 13
Link terkumpul di hewania_links.txt: 14
Link terkumpul di hewania_links.txt: 15
Link terkumpul di hewania_links.txt: 16
Link terkumpul di hewania_links.txt: 17
Link terkumpul di hewania_links.txt: 18
Link terkumpul di hewania_links.txt: 19
Halaman 2 untuk hewania_links.txt selesai.
Link terkumpul di hewania_links.txt: 20
Link terkumpul di hewania_links.txt: 21
Link terkumpul di hewania_links.txt: 22
Link terkumpul di hewania_links.txt: 23
Lin

## Scraping

In [2]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import time

# Fungsi asynchronous untuk melakukan scraping dari URL
async def scrape_link(session, link, writer, counter, semaphore):
    try:
        # Batasi jumlah koneksi bersamaan
        async with semaphore:
            async with session.get(link, timeout=10) as response:
                content = await response.text()
                soup = BeautifulSoup(content, 'html.parser')

                # Ambil judul
                title_element = soup.find('h1', class_='bt_bb_headline_tag')
                title = title_element.get_text(strip=True) if title_element else 'No title'

                # Ambil tanggal
                date_element = soup.find('span', class_='btArticleDate')
                date = date_element.get_text(strip=True) if date_element else 'No date'

                # Ambil konten
                content_element = soup.find('div', class_='btArticleContent')
                
                # Ambil hanya teks dari elemen <p> dan abaikan hyperlink di dalamnya
                if content_element:
                    paragraphs = content_element.find_all('p')
                    content = ' '.join([p.get_text(strip=True, separator=' ') for p in paragraphs if not p.find('a')])
                else:
                    content = 'No content'

                # Ambil URL gambar
                image_element = soup.find('div', class_='btMediaBox').find('img') if soup.find('div', class_='btMediaBox') else None
                image_url = image_element['src'] if image_element else 'No image'

                # Skip jika tidak ada konten
                if content == 'No content':
                    return

                # Menulis data ke CSV dengan menambahkan kolom gambar
                writer.writerow([link, title, date, content, image_url])
                counter['count'] += 1  # Increment count

                # Print setiap 1 link yang di-scrape
                if counter['count'] % 1 == 0:
                    print(f"{counter['count']} link berhasil di-scrape.")
    
    except asyncio.TimeoutError:
        print(f"Timeout error pada link: {link}")
    except Exception as e:
        print(f"Error pada link {link}: {e}")

# Fungsi utama untuk melakukan scraping dari satu file
async def scrape_from_links_async(input_file, csv_file):
    start_time = time.time()  # Mulai menghitung waktu

    with open(input_file, 'r', encoding='utf-8') as file:
        links = file.readlines()

    async with aiohttp.ClientSession() as session:
        # Membuka file CSV untuk menulis hasil scraping
        with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)  # Quote all fields
            writer.writerow(['Link', 'Judul', 'Tanggal', 'Content', 'Image URL'])  # Header CSV dengan kolom gambar
            
            # Daftar untuk menyimpan semua task
            tasks = []
            counter = {'count': 0}  # Counter untuk jumlah link yang di-scrape

            # Batasi jumlah koneksi bersamaan
            semaphore = asyncio.Semaphore(10)  # Batasi 10 koneksi bersamaan

            for link in links:
                link = link.strip()
                task = asyncio.create_task(scrape_link(session, link, writer, counter, semaphore))
                tasks.append(task)

            # Menunggu semua task selesai
            await asyncio.gather(*tasks)

    end_time = time.time()  # Akhir waktu
    duration = end_time - start_time
    print(f"Scraping selesai dalam {duration:.2f} detik.")

# Fungsi untuk menjalankan scraping pada file tunggal
def scrape():
    input_file = "hewania_links.txt"  # Ganti dengan file input yang sesuai
    output_file = "scraped_hewania_articles.csv"  # Nama file output CSV

    # Cek apakah event loop sudah berjalan
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # Tidak ada event loop yang berjalan
        asyncio.run(scrape_from_links_async(input_file, output_file))
    else:  # Event loop sedang berjalan
        loop.create_task(scrape_from_links_async(input_file, output_file))

# Memanggil fungsi untuk menjalankan scraping
scrape()


1 link berhasil di-scrape.
2 link berhasil di-scrape.
3 link berhasil di-scrape.
4 link berhasil di-scrape.
5 link berhasil di-scrape.
6 link berhasil di-scrape.
7 link berhasil di-scrape.
8 link berhasil di-scrape.
9 link berhasil di-scrape.
10 link berhasil di-scrape.
11 link berhasil di-scrape.
12 link berhasil di-scrape.
13 link berhasil di-scrape.
14 link berhasil di-scrape.
15 link berhasil di-scrape.
16 link berhasil di-scrape.
17 link berhasil di-scrape.
18 link berhasil di-scrape.
19 link berhasil di-scrape.
20 link berhasil di-scrape.
21 link berhasil di-scrape.
22 link berhasil di-scrape.
23 link berhasil di-scrape.
24 link berhasil di-scrape.
25 link berhasil di-scrape.
26 link berhasil di-scrape.
27 link berhasil di-scrape.
28 link berhasil di-scrape.
29 link berhasil di-scrape.
30 link berhasil di-scrape.
31 link berhasil di-scrape.
32 link berhasil di-scrape.
33 link berhasil di-scrape.
34 link berhasil di-scrape.
35 link berhasil di-scrape.
36 link berhasil di-scrape.
3

## Preprocessing

### Remove punctuations dan angka

In [3]:
import re
import csv
import pandas as pd

# Membaca file hasil scraping dengan pembatas tanda kutip ganda
data = pd.read_csv("scraped_hewania_articles.csv", quoting=csv.QUOTE_ALL)

# Fungsi untuk membersihkan tanda baca, angka, dan nomor
def clean_text(text):
    # Mengecek apakah teks valid (bukan NaN)
    if isinstance(text, str):  
        text = re.sub(r'[^\w\s]', '', text)  # Menghapus tanda baca
        text = re.sub(r'\d+', '', text)  # Menghapus angka dan nomor
        text = re.sub(r'\s+', ' ', text).strip()  # Menghapus spasi berlebih
        return text
    return text  # Kembalikan teks apa adanya jika bukan string

# Terapkan fungsi pada kolom 'Content' dan 'Judul'
data['Judul_Cleaned'] = data['Judul'].apply(clean_text)
data['Content_Cleaned'] = data['Content'].apply(clean_text)

# Simpan hasilnya kembali ke file CSV tanpa mengubah kolom lama
data.to_csv("cleaned_hewania_articles.csv", index=False, quoting=csv.QUOTE_ALL)

print("Preprocessing Remove punctuations dan nomor, angka selesai dan file hasil telah disimpan.")


Preprocessing Remove punctuations dan nomor, angka selesai dan file hasil telah disimpan.


## Menghilangkan stopword

In [4]:
import pandas as pd
import re

# Membaca file hasil scraping
data = pd.read_csv("cleaned_hewania_articles.csv", quoting=csv.QUOTE_ALL)

# Membaca daftar stopwords dari file stopword.txt
with open('stopword.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# Fungsi untuk menghapus stopwords dari teks
def remove_stopwords(text):
    if isinstance(text, str):  # Pastikan input adalah string
        words = text.split()
        cleaned_text = ' '.join([word for word in words if word.lower() not in stopwords])
        return cleaned_text
    return text  # Jika bukan string, kembalikan nilai aslinya

# Terapkan fungsi ke kolom 'Judul' dan 'Content'
data['Judul_Cleaned'] = data['Judul_Cleaned'].apply(remove_stopwords)
data['Content_Cleaned'] = data['Content_Cleaned'].apply(remove_stopwords)


# Menyimpan hasil yang sudah diolah ke file CSV baru
data.to_csv("cleaned_hewania_articles.csv", index=False, quoting=csv.QUOTE_ALL)

print("Preprocessing Remove Stopword selesai dan file hasil telah disimpan.")


Preprocessing Remove Stopword selesai dan file hasil telah disimpan.


### Case Folding

In [5]:
import pandas as pd
import csv

# Membaca file hasil scraping
data = pd.read_csv("cleaned_hewania_articles.csv", quoting=csv.QUOTE_ALL)

# Melakukan case folding (mengubah ke huruf kecil) pada kolom Judul dan Content
data['Judul_Cleaned'] = data['Judul_Cleaned'].str.lower()
data['Content_Cleaned'] = data['Content_Cleaned'].str.lower()

# Menyimpan kembali file CSV dengan perubahan pada kolom Judul dan Content
data.to_csv("cleaned_hewania_articles.csv", index=False, quoting=csv.QUOTE_ALL)

print("Case folding telah selesai dan hasilnya telah ditimpa pada kolom Judul dan Content.")


Case folding telah selesai dan hasilnya telah ditimpa pada kolom Judul dan Content.


### Stemming

In [6]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import csv
import time 

# Membaca file 
data = pd.read_csv("cleaned_hewania_articles.csv", quoting=csv.QUOTE_ALL)

# Membuat objek Stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk melakukan stemming pada kolom tertentu
def apply_stemming(column_data):
    # Menggunakan list comprehension untuk lebih cepat daripada apply
    return column_data.apply(lambda x: stemmer.stem(x) if isinstance(x, str) else x)

# Memeriksa dan memastikan kolom yang digunakan berisi string
data['Judul_Cleaned'] = data['Judul_Cleaned'].astype(str)
data['Content_Cleaned'] = data['Content_Cleaned'].astype(str)

# Mulai pengukuran waktu
start_time = time.time()

# Melakukan stemming pada kolom Judul_Cleaned dan Content_Cleaned
data['Judul_Cleaned'] = apply_stemming(data['Judul_Cleaned'])
print("Kolom judul berhasil")
data['Content_Cleaned'] = apply_stemming(data['Content_Cleaned'])
print("Kolom konten berhasil")

# Menyimpan kembali file CSV dengan perubahan pada kolom Judul dan Content
data.to_csv("cleaned_hewania_articles_stemmed.csv", index=False, quoting=csv.QUOTE_ALL)

# Menghitung waktu yang telah berlalu
end_time = time.time()
execution_time = end_time - start_time

print(f"Stemming telah selesai dan hasilnya telah disimpan pada file baru.")
print(f"Waktu yang dibutuhkan untuk proses stemming: {execution_time:.2f} detik.")


Kolom judul berhasil
Kolom konten berhasil
Stemming telah selesai dan hasilnya telah disimpan pada file baru.
Waktu yang dibutuhkan untuk proses stemming: 573.18 detik.


### Tokenisasi

In [7]:
import pandas as pd
import csv

# Membaca file
data = pd.read_csv("cleaned_hewania_articles_stemmed.csv", quoting=csv.QUOTE_ALL)

# Fungsi untuk melakukan tokenisasi sederhana
def apply_tokenization(column_data):
    return column_data.apply(lambda x: x.split() if isinstance(x, str) else x)

# Tokenisasi pada kolom Judul_Cleaned dan Content_Cleaned
data['Judul_Cleaned'] = apply_tokenization(data['Judul_Cleaned'])
data['Content_Cleaned'] = apply_tokenization(data['Content_Cleaned'])

# Menyimpan kembali file CSV dengan perubahan pada kolom Judul_Cleaned dan Content_Cleaned
data.to_csv("cleaned_hewania_articles_tokenized.csv", index=False, quoting=csv.QUOTE_ALL)

print("Tokenisasi selesai dan hasilnya telah disimpan pada file baru.")


Tokenisasi selesai dan hasilnya telah disimpan pada file baru.


## Membuat Search Engine menggunakan algoritma Cosine Similarity & Jaccard Similarity

In [ ]:
#membuat file bernama app.py dengan isi sebagai berikut:

from flask import Flask, render_template, request
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

app = Flask(__name__)

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load data
df = pd.read_csv("cleaned_hewania_articles_tokenized.csv")
df.fillna("", inplace=True)

# Gabungkan kolom judul dan konten untuk perhitungan
df["combined"] = df["Judul_Cleaned"] + " " + df["Content_Cleaned"]

# Load stopwords
with open("stopword.txt", "r") as file:
    stopwords = set(file.read().splitlines())

# Fungsi preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Hilangkan angka dan tanda baca
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    # Tokenisasi dan hapus stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return " ".join(tokens)

# Fungsi untuk melakukan stemming pada query menggunakan Sastrawi
def stem_query(query):
    return stemmer.stem(query)  # Terapkan stemming pada query

# Fungsi Cosine Similarity
def get_cosine_similarity(query, top_n=20):
    query = preprocess_text(query)
    query = stem_query(query)  # Terapkan stemming pada query
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([query] + df["combined"].tolist())
    cosine_sim = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
    
    results = []
    for idx, cosine_score in enumerate(cosine_sim):
        results.append({
            "index": idx,
            "title": df.loc[idx, "Judul"],
            "link": df.loc[idx, "Link"],
            "date": df.loc[idx, "Tanggal"],
            "content": df.loc[idx, "Content"][:200] + "...",
            "image": df.loc[idx, "Image URL"],
            "cosine_similarity": cosine_score,
        })
    
    results = sorted(results, key=lambda x: x["cosine_similarity"], reverse=True)
    return results[:top_n]

# Fungsi Jaccard Similarity
def get_jaccard_similarity(query, top_n=20):
    query = preprocess_text(query)  # Preprocessing untuk query
    query = stem_query(query)  # Terapkan stemming pada query
    query_tokens = set(query.split())  # Tokenkan query
    
    results = []
    for idx, row in df.iterrows():
        # Preprocessing hanya dilakukan di Jaccard
        doc_tokens = set(preprocess_text(row["combined"]).split())  # Preprocess langsung
        
        # Hitung Jaccard Similarity
        intersection = len(query_tokens & doc_tokens)
        union = len(query_tokens | doc_tokens)
        jaccard_score = intersection / union if union != 0 else 0
        
        results.append({
            "index": idx,
            "title": row["Judul"],
            "link": row["Link"],
            "date": row["Tanggal"],
            "content": row["Content"][:200] + "...",
            "image": row["Image URL"],
            "jaccard_similarity": jaccard_score,
        })
    
    results = sorted(results, key=lambda x: x["jaccard_similarity"], reverse=True)
    return results[:top_n]

# Route index - Menampilkan artikel terbaru
@app.route('/')
def index():
    # Ambil artikel terbaru (misalnya 3 artikel terbaru berdasarkan tanggal)
    latest_articles = df.sort_values(by='Tanggal', ascending=False).head(3)
    
    # Konversi DataFrame menjadi list of dictionaries
    latest_articles_dict = latest_articles.to_dict(orient='records')
    
    # Render template dengan data artikel terbaru
    return render_template('index.html', latest_articles=latest_articles_dict)

# Route search
@app.route('/search', methods=['GET', 'POST'])
def search():
    query = request.form.get('query', '') if request.method == 'POST' else request.args.get('query', '')
    algorithm = request.form.get('algorithm', 'cosine') if request.method == 'POST' else request.args.get('algorithm', 'cosine')
    page = int(request.args.get('page', 1))
    per_page = 10
    
    if algorithm == 'cosine':
        all_results = get_cosine_similarity(query, top_n=30)
    elif algorithm == 'jaccard':
        all_results = get_jaccard_similarity(query, top_n=30)
    else:
        all_results = []
    
    # Filter out results with a score of 0
    filtered_results = [result for result in all_results if (result.get('cosine_similarity', 0) > 0 or result.get('jaccard_similarity', 0) > 0)]
    
    total_results = len(filtered_results)
    total_pages = (total_results + per_page - 1) // per_page
    page = max(1, min(page, total_pages))
    start = (page - 1) * per_page
    end = start + per_page
    results = filtered_results[start:end]
    
    return render_template(
        'results.html',
        query=query,
        results=results,
        algorithm=algorithm,
        page=page,
        total_pages=total_pages,
        total_results=total_results
    )

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
!python app.py